### Импорт

In [5]:
# импортируем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# импортируем данные
df = pd.read_csv(r"C:\Users\allek\OneDrive\Рабочий стол\summer_train\train_data.csv")

In [7]:
# df = pd.read_csv(r"C:\Users\allek\OneDrive\Рабочий стол\summer_train\git_project\Diabetes-ML-From-Scratch\raw_data\diabetes.csv")

### Осмотр данных

In [8]:
# df['Glucose'] = np.where(df['Glucose'] == 0, df['Glucose'].mean(), df['Glucose'])
# df['BloodPressure'] = np.where(df['BloodPressure'] == 0, df['BloodPressure'].mean(), df['BloodPressure'])
# df['SkinThickness'] = np.where(df['SkinThickness'] == 0, df['SkinThickness'].mean(), df['SkinThickness'])
# df['Insulin'] = np.where(df['Insulin'] == 0, df['Insulin'].mean(), df['Insulin'])
# df['BMI'] = np.where(df['BMI'] == 0, df['BMI'].mean(), df['BMI'])

In [9]:
df.head()

,Pregnancies,SkinThickness,Insulin,DiabetesPedigreeFunction,Age,Outcome,HOMA_IR,BMI_AGE_BOND,Glucose_category_norm,Glucose_category_prediabetes,Glucose_category_diabetes,BMI_category_underweight,BMI_category_norm,BMI_category_overweight,BMI_category_obese,Blood_Pressure_category_norm,Blood_Pressure_category_prehypertension,Blood_Pressure_category_hypertension
0,-0.528766,0.005299,0.015212,-0.490335,-1.035096,0,-0.466783,-0.915001,True,False,False,False,False,False,True,True,False,False
1,1.611193,-0.550407,0.015212,2.413062,1.485889,1,-0.165841,0.817008,False,True,False,False,False,True,False,False,True,False
2,-0.834475,-1.154976,-0.878724,0.548713,-0.948166,0,-0.566145,-1.028785,False,False,True,False,False,True,False,True,False,False
3,-1.140183,0.005299,0.015212,-0.638770,2.789848,0,0.360808,0.849012,False,False,True,False,True,False,False,True,False,False
4,0.694068,1.021472,2.718062,-0.686270,1.138167,1,2.083261,2.512501,False,False,True,False,False,False,True,False,True,False


### Реализация логистической регресии

In [11]:
def sigmoid(z):
    z = np.asarray(z, dtype=float)
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))

In [12]:
def binary_cross_entropy_vectorized(p_true, p_pred):
    """
    Векторизованная бинарная кросс-энтропия
    p_true: one-hot encoded метки [[0,1], [1,0], ...]
    p_pred: предсказанные вероятности [0.8, 0.2, ...]
    """
    # Защита от log(0)
    p_pred = np.clip(p_pred, 1e-15, 1 - 1e-15)
    
    # Векторизованное вычисление
    # p_true[i][1] соответствует y=1, p_true[i][0] соответствует y=0
    loss = -np.mean(p_true[:, 1] * np.log(p_pred) + p_true[:, 0] * np.log(1 - p_pred))
    
    return loss

In [13]:
X_train = np.array(df.drop('Outcome', axis=1))
Y_train = np.array(df['Outcome'])

In [39]:
# инициализация весов
w = np.ones(X_train.shape[-1])
b = 0.01
learning_rate = 0.7
k = 612

In [40]:
for epoch in range(700):
    # выборка для SGD
    rand_index = np.random.choice(X_train.shape[0], k, replace=False)
    sample_X = X_train[rand_index]
    sample_Y = Y_train[rand_index]

    # подсчет вероятности
    linear_model = w @ sample_X.T + b
    pred = sigmoid(linear_model)

    # потери
    y_one_hot = np.eye(2)[sample_Y]
    loss = binary_cross_entropy_vectorized(y_one_hot, pred)
    if epoch % 100 == 0: print(loss)

    # веса
    dz = pred - sample_Y
    dw = (1 / k) * np.dot(sample_X.T, dz)
    db = (1 / k) * np.sum(dz)

    # Преобразуйте dw и db к numpy array с правильным типом
    w -= learning_rate * np.asarray(dw, dtype=float)
    b -= learning_rate * np.asarray(db, dtype=float)

1.7094167492063734
0.47225265719318077
0.4706398604964984
0.47042796848750024
0.4679463603341514
0.4691718807195346
0.469877531455832


In [41]:
prediction = np.where(sigmoid(w @ X_train.T + b) > 0.5, 1, 0)
np.mean(prediction == Y_train)

np.float64(0.7752442996742671)

In [42]:
df_test = pd.read_csv(r"C:\Users\allek\OneDrive\Рабочий стол\summer_train\test_data.csv")
X_test = np.array(df_test.drop('Outcome', axis=1))
Y_test = np.array(df_test['Outcome'])

In [43]:
pred_test = np.where(sigmoid(w @ X_test.T + b) > 0.5, 1, 0)
np.mean(pred_test == Y_test)

np.float64(0.7402597402597403)